In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
RANDOM_SEED = 0x0

# Lab 03: Gradient Descent and Polynomial Regression

For the first task, we will work with synthetic univariate data. This is the same data as in the previous Lab.
We generate $100$ features $x_i \in [-1, 1]$ as `x` and a regression target `y1`.

In [ ]:
data_rng = np.random.default_rng(RANDOM_SEED)
n = 100
x = 2 * data_rng.random(n) - 1  # create n points between -1 and 1

# setup synthetic linear data
true_offset = 0.5
true_slope = 1.25
noise = data_rng.normal(loc=0., scale=0.25, size=(n,))

y1 = true_offset + true_slope * x + noise

This is what the data looks like:

In [ ]:
plt.scatter(x, y1)
plt.xlabel("x")
plt.ylabel("y1")
plt.show()

### 📢 Task 1A (3 points)📢

In class you have seen that, instead of using the closed form solutions, we can also estimate the parameters $\theta_i$ of the linear regression models by using Gradient Descent.

For the univariate linear regression model, the stochastic gradient descent updates (from step t to t+1) look like this:
* $\theta_{0}^{(t+1)} = \theta_{0}^{(t)} - \alpha (\theta_{0}^{(t)} + \theta_{1}^{(t)} x_t - y_t)$
* $\theta_{1}^{(t+1)} = \theta_{1}^{(t)} - \alpha (\theta_{0}^{(t)} + \theta_{1}^{(t)} x_t - y_t) x_t$

Here $\alpha$ is the learning rate, and $(x_t, y_t)$ is the data point sampled
at time $t$.


In the following cell, implement the `.fit` and `.predict` methods:
* In the `.predict` method, apply the model to the input `x`.
* In the `.fit` method, implement the update equations for
$\theta_0$ and $\theta_1$.

In [ ]:
class SGDUnivariateLinearRegression:

  def __init__(self):
    self.theta_0: float = 0.
    self.theta_1: float = 0.
    self.rng = np.random.default_rng(RANDOM_SEED)

  def predict(self, x):
    y = self.theta_0 + self.theta_1 * x
    return y  # TODO



  def fit(self, x, y, n_iter: int = 100, learning_rate: float = 1.0):
    for t in range(n_iter):
      sample_ix = self.rng.integers(0, len(x))

      xt = x[sample_ix]
      yt = y[sample_ix]

      t0 = self.theta_0
      t1 = self.theta_1

      self.theta_0 = t0 - learning_rate * (t0 + t1 *xt - yt)
      self.theta_1 = t1 - learning_rate * (t0 + t1 *xt - yt) * xt
      # TODO: update self.theta_0 and self.theta_1 SIMULTANEOUSLY (!!!) according to their update equations

    return self

📢 **<mark>On Moodle</mark>** 📢

* Hand in your code from the cell above.

### 📢 Task 1B (2 points)📢

Run SGD for `x` and the target `y1` and compute the mean squared error (MSE).
The MSE is defined as: $\frac{1}{n}\sum_{i=1}^{n} (\hat{y}_i - y_i)^2$, where
$\hat{y}$ are the model predictions.

* Create an instance of the class `SGDUnivariateLinearRegression`
* fit the model using its `.fit` method
* get the predicted values, using `.predict`
* implement the `mse` function
* compute the MSE of your predictions

In [ ]:
def mse(y_pred, y_true):
  #TODO
  sum = 0
  for i in range(len(y_pred)):
    sum += (y_pred[i] - y_true[i])**2

  return sum / len(y_pred)

In [ ]:
# TODO: implement fit an predict
model = SGDUnivariateLinearRegression()

# model.fit(x, y1, n_iter=20000, learning_rate=0.001)
model.fit(x, y1)
y_pred = model.predict(x)
error = mse(y_pred, y1)
print("MSE: ", error)

📢 **<mark>On Moodle</mark>** 📢
* Report the resulting Mean Squared Error

If you want to plot the resulting model, you can use the following function:

In [ ]:
def plot_model(x, y_pred, y_true, title):
  plt.scatter(x, y_true)
  plt.plot(x, y_pred, c="r")
  plt.xlabel("x")
  plt.ylabel("y")
  plt.title(title)
  plt.show()

plot_model(x, y_pred, y1, "Model of Gradient Descent for y1")

### 📢 Task 1C (4 points) 📢

You will now plot the learning curves of gradient descent for different learning rates $\alpha$.
A learning curves shows how a model's performance changes with increasing number of update steps.
In our case we will plot the model's MSE as a function of the number of update
steps `n_iter` for different values of `learning_rate`.


For comparison, we first implement the closed form for univariate regression from previous lab:

In [ ]:
class UnivariateLinearRegression:

  def __init__(self):
    self.theta_0: float = 0.
    self.theta_1: float = 0.

  def predict(self, x):
    return self.theta_0 + self.theta_1 * x

  def fit(self, x, y):
    mean_x = np.mean(x)
    mean_y = np.mean(y)

    cent_x = x - mean_x
    cent_y = y - mean_y

    theta_1 = np.sum(cent_x * cent_y) / np.sum(cent_x * cent_x)
    theta_0 = mean_y - theta_1 * mean_x

    self.theta_1 = theta_1
    self.theta_0 = theta_0

    return self

In [ ]:
closed_form = UnivariateLinearRegression()
y_pred = closed_form.fit(x, y1).predict(x)


plot_model(x, y_pred, y1, "Model of Gradient Descent for y1")


In the following cell we setup most of the scaffold to create learning curves. A learning curve contains the MSE for different numbers of iterations, i.e. different number of updates of the parameters in θ.

We will draw learning curves for different values of learning rate α (don't get confused with the duplicate usage of "learning" here. The first refers to the improvement of MSE over number of iterations, while the second influences how much we change values in θ in each step).

Follow the instructions in the comments to finish the plots.

In [ ]:
n_iters = [50, 100, 200, 500, 1000, 2000]
learning_rates = [1., .1, .01]

# we plot the MSE achieved by the closed form model as a reference
closed_form = UnivariateLinearRegression()
closed_form.fit(x, y1)
mse_base = mse(y_pred=closed_form.predict(x), y_true=y1)
plt.plot(n_iters, np.ones_like(n_iters) * mse_base, label="closed form", linestyle='--', c='b')

model = SGDUnivariateLinearRegression()
for alpha in learning_rates:
  mses = []
  for n_iter in n_iters:
    # fit a SGDUnivariateLinearRegression model using n_iter=n_iter and
    # learning_rate=alpha
    # compute its mse and append the mse value to the mses list


    model.fit(x, y1, n_iter=n_iter, learning_rate=alpha)
    y_pred = model.predict(x)
    mse_ = mse(y_pred, y1)  # replace with mse calculation
    mses.append(mse_)
  plt.plot(n_iters, mses, label=f"alpha = {alpha:.2f}")

plt.xlabel("n_iter")
plt.ylabel("MSE")
plt.legend()
plt.show()

📢 **<mark>On Moodle</mark>** 📢

* The final plot containing the 3 learning curves
* A short (2-3 sentences) interpretation of the curves: why do you think they look the way they do? Can you draw any conclusions?

In case you were not able to arrive at the final plot:

* include screenshots of the code you wrote so we can assign partial credit




# TASK 2: Polynomial Regression

We now switch to polynomial regression. First, let's create and explore some new synthetic data for this task:

In [ ]:
# set the random seed to an integer, so that everyone has the same data to work with
np.random.seed(seed=RANDOM_SEED)
# create predictor variable that has a standard normal distribution, and reshape it in order to use it for the model training
x = np.random.normal(0, 1, 100).reshape(-1, 1)
# create target variable
y = 3*x**3 + 2*x**2 + x + np.random.normal(0, 10, 100).reshape(-1, 1)

Visualise the data

In [ ]:
plt.scatter(x, y)
plt.show()

First, we try to fit a *Linear* Regression model to the data.

For this, we first split the data into a train set and a test set (80% train, 20% test data).

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=RANDOM_SEED)

## Task 2A (1 point)
1. Apply Linear Regression on the data and predict y values for training set as well for the test set.

In [ ]:
from sklearn.linear_model import LinearRegression
# TODO
model = LinearRegression()
model.fit(X_train, y_train)


y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)


2. Calculate MSE for training as well as for test data.

In [ ]:

from sklearn.metrics import mean_squared_error

# TODO
mse_train = mean_squared_error(y_train, y_train_predict)
mse_test = mean_squared_error(y_test, y_test_pred)
print(f"MSE of training data: {mse_train}")
print(f"MSE of test data: {mse_test}")

3. Visualize the model. Plot all the data points als well as the regression model when applied to the training set and to the test set (in plt.scatter, you can set the color of points e.g. to red using " color='r' ").

In [ ]:
# TODO
# Scatter plots of data
plt.scatter(X_train, y_train, color="r", label="Train data")
plt.scatter(X_test, y_test, color="b", label="Test data")

# To plot regression lines, we sort X for a clean line
X_line = np.linspace(min(X_train.min(), X_test.min()),
                     max(X_train.max(), X_test.max()), 100).reshape(-1, 1)

y_line = model.predict(X_line)

# Regression line
plt.plot(X_line, y_line, color="k", label="Regression line")

plt.xlabel("X")
plt.ylabel("y")
plt.legend()
plt.title("Linear Regression: Train & Test Visualization")
plt.show()

📢 **<mark>On Moodle</mark>** 📢

Upload
* your scatter plot

## Task 2B (2 points)

You will now investigate how well polynomial regression with a polynomial of degree 2 can solve the task.




Recall that for polynomial regression, we introduce artificial variables $z_i$ and a new hypothesis $h(z) = \theta_0 z_0 + \theta_1 z_1 + \theta_2 z_2 + \theta_3 z_3 ...$, where each $z_i$ is a polynomial in $x$. One of the simplest hypothesis is quadratic, where we have $h(z) = \theta_0 z_0 + \theta_1 z_1 + \theta_2 z_2$ with $z_0 = 1, z_1 = x$ and $z_2 = x^2$.

In the following cell, implement polynomial regression with the quadratic hypothesis above, and plot the result.

In [ ]:
# TODO
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=2)

X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

poly_model = LinearRegression()
poly_model.fit(X_train_poly, y_train)

y_train_pred = poly_model.predict(X_train_poly)
y_test_pred = poly_model.predict(X_test_poly)

mse_train_poly = mean_squared_error(y_train, y_train_pred)
mse_test_poly = mean_squared_error(y_test, y_test_pred)

X_line = np.linspace(min(X_train.min(), X_test.min()),
                     max(X_train.max(), X_test.max()), 200).reshape(-1, 1)
X_line_poly = poly.transform(X_line)
y_line = poly_model.predict(X_line_poly)

Plot the result


In [ ]:
# TODO
plt.scatter(X_train, y_train, color="r", label="Train data")
plt.scatter(X_test, y_test, color="b", label="Test data")
plt.plot(X_line, y_line, color="k", label="Polynomial degree 2 fit")

plt.xlabel("X")
plt.ylabel("y")
plt.legend()
plt.title("Polynomial Regression (degree=2)")
plt.show()

Instead of implementing the polynomial transformation by hand, we can use [PolynomialFeatures.fit_transform](https://scikit-learn.org/stable/modules/preprocessing.html#polynomial-features) from sklearn, which generates the new features automatically.

1. Transform the data accordingly to describe polynomial distribution of degree=2
2. Train a Linear Regression model on polynomial data
3. Make predictions for training data
4. Make predictions for test data
5. Calculate MSE for training as well as test data

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

# TODO

print(f"MSE of training data: {mse_train_poly}")
print(f"MSE of test data: {mse_test_poly}")

Did it perform better than Linear Regression? Visualize the results.

In [ ]:
# TODO
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error

# Fit linear model
lin = LinearRegression().fit(X_train, y_train)
ytr_lin = lin.predict(X_train)
yte_lin = lin.predict(X_test)

# Fit degree-2 polynomial model
poly = PolynomialFeatures(degree=2, include_bias=True)
Xtr2 = poly.fit_transform(X_train)
Xte2 = poly.transform(X_test)
poly2 = LinearRegression().fit(Xtr2, y_train)
ytr_poly2 = poly2.predict(Xtr2)
yte_poly2 = poly2.predict(Xte2)

# MSEs
lin_tr_mse = mean_squared_error(y_train, ytr_lin)
lin_te_mse = mean_squared_error(y_test, yte_lin)
poly2_tr_mse = mean_squared_error(y_train, ytr_poly2)
poly2_te_mse = mean_squared_error(y_test, yte_poly2)

print(f"Linear -> Train MSE: {lin_tr_mse:.4f} | Test MSE: {lin_te_mse:.4f}")
print(f"Poly d=2 -> Train MSE: {poly2_tr_mse:.4f} | Test MSE: {poly2_te_mse:.4f}")
print("Degree-2 better on TEST? ", "YES" if poly2_te_mse < lin_te_mse else "NO")

# Visualization
X_line = np.linspace(min(X_train.min(), X_test.min()),
                     max(X_train.max(), X_test.max()), 400).reshape(-1, 1)
y_line_lin = lin.predict(X_line)
y_line_poly2 = poly2.predict(poly.transform(X_line))

plt.figure(figsize=(8,5))
plt.scatter(X_train, y_train, color='r', alpha=0.6, label='Train')
plt.scatter(X_test, y_test, color='b', alpha=0.6, label='Test')
plt.plot(X_line, y_line_lin, label='Linear fit', linewidth=2)
plt.plot(X_line, y_line_poly2, label='Polynomial fit (degree=2)', linewidth=2)
plt.xlabel("X"); plt.ylabel("y"); plt.title("Linear vs Degree-2 Polynomial Regression")
plt.legend(); plt.tight_layout(); plt.show()


📢 **<mark>On Moodle</mark>** 📢

Upload your scatter plot


## Task 2C (4 point)
Finally, we investigate the influence of the polynomial degree on the results. Consider degrees in range(0, 11). Plot MSE (on training as well as test data) depending on the polynomial degree.

In [ ]:
# TODO

📢 **<mark>On Moodle</mark>** 📢

* Upload your plot of MSE for training and test data

* Answer the following three questions in a text:
1. What is the optimal value of the polynomial degrees?
2. Do the values of MSE training and MSE test behave similarly?
3. What happens in the models with polynomial degrees >= 8?


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

degrees = range(0, 11)
train_mses, test_mses = [], []

for d in degrees:
    poly = PolynomialFeatures(degree=d, include_bias=True)
    Xtr = poly.fit_transform(X_train)
    Xte = poly.transform(X_test)

    model = LinearRegression()
    model.fit(Xtr, y_train)

    ytr_pred = model.predict(Xtr)
    yte_pred = model.predict(Xte)

    train_mses.append(mean_squared_error(y_train, ytr_pred))
    test_mses.append(mean_squared_error(y_test, yte_pred))

plt.plot(degrees, train_mses, marker='o', label='Train MSE')
plt.plot(degrees, test_mses, marker='o', label='Test MSE')
plt.xticks(degrees)
plt.xlabel('Polynomial degree')
plt.ylabel('MSE')
plt.legend()
plt.title('Train/Test MSE vs Polynomial Degree')
plt.show()

best_deg = degrees[int(np.argmin(test_mses))]
print("Best degree by test MSE:", best_deg)
